In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from myagents.prompts import agent1_prompt,agent2_prompt,agent3_prompt



from tools.search_tool import search_content


llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# llm = ChatOpenAI(model="gpt-4o-mini")
tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}

# --- Define Agent 2: Content Agent ---
content_agent_prompt = SystemMessage(content=agent2_prompt)
def content_agent(state: MessagesState):
    return {"messages": [llm.invoke([content_agent_prompt] + state["messages"])]}

# --- Define Agent 3: Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}

# --- Build Graph ---
graph = StateGraph(MessagesState)

graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))
graph.add_node("content_agent", content_agent)
graph.add_node("tune_agent", tune_agent)


#  Use tools_condition to decide when to call the tool
graph.add_edge(START,"search_agent")
graph.add_conditional_edges(
    "search_agent",
    tools_condition,
    {"tools": "tools", "__end__": "content_agent"}
)

# After tools run → back to search_agent
graph.add_edge("tools", "search_agent")

graph.add_edge("search_agent","content_agent")

graph.add_edge("content_agent","tune_agent")

# tune Agent → END
graph.add_edge("tune_agent", END)

# --- Compile Agent ---
agent = graph.compile()
agent


: 

In [ ]:

import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from myagents.prompts import agent1_prompt, agent2_prompt, agent3_prompt


from tools.search_tool import search_content

# --- Initialize LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# llm = ChatOpenAI(model="gpt-4o-mini")

tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}


# === Specialized Content Agents ===
def blog_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write an engaging, SEO-friendly blog post.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def article_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a well-researched, formal article.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def summary_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Summarize the topic in concise bullet points.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def email_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a professional email draft.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def script_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Create a creative video or podcast script.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def social_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a catchy social media caption or post.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def general_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a detailed explanation.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}


# --- Define Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}


# --- Helper Function: Decide Which Agent to Use ---
def select_content_agent(state: MessagesState):
    """
    Decide which content agent to use based on `content_type` inside the user's message.
    """
    messages = state["messages"]
    content_type = None

    for msg in messages:
        if isinstance(msg, HumanMessage) and isinstance(msg.content, dict):
            content_type = msg.content.get("content_type", "").lower()
            break

    mapping = {
        "blog": "blog_agent",
        "article": "article_agent",
        "summary": "summary_agent",
        "email": "email_agent",
        "script": "script_agent",
        "social": "social_agent",
    }

    # Default to general_agent if not found
    return mapping.get(content_type, "general_agent")


# --- Build Graph ---
graph = StateGraph(MessagesState)
graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))

# Add all specialized content agents
graph.add_node("blog_agent", blog_agent)
graph.add_node("article_agent", article_agent)
graph.add_node("summary_agent", summary_agent)
graph.add_node("email_agent", email_agent)
graph.add_node("script_agent", script_agent)
graph.add_node("social_agent", social_agent)
graph.add_node("general_agent", general_agent)

graph.add_node("tune_agent", tune_agent)

# --- Graph Connections ---
graph.add_edge(START, "search_agent")

# Conditional path after search
graph.add_conditional_edges("search_agent", tools_condition, {"tools": "tools", "__end__": select_content_agent})
graph.add_edge("tools", "search_agent")

# After chosen content agent → tuning → END
for node in ["blog_agent", "article_agent", "summary_agent", "email_agent", "script_agent", "social_agent", "general_agent"]:
    graph.add_edge(node, "tune_agent")

graph.add_edge("tune_agent", END)

agent = graph.compile()

agent

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI


from tools.search_tool import search_content
from myagents.prompts import agent1_prompt, agent3_prompt  # removed agent2_prompt

# --- Initialize LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# llm = ChatOpenAI(model="gpt-4o-mini")

tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}


# === Specialized Content Agents ===
def blog_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a professional **blog writer**. "
        "Write an engaging, SEO-optimized blog post with:\n"
        "- A catchy title\n"
        "- Subheadings\n"
        "- Conversational tone\n"
        "- Conclusion\n"
        "Make it informative and easy to read."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def article_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a **research-based article writer**. "
        "Write a well-structured, factual, and formal article including:\n"
        "- Clear introduction\n"
        "- Supporting evidence and citations (if relevant)\n"
        "- Conclusion summarizing insights."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def summary_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a **content summarizer**. "
        "Summarize the provided topic into concise bullet points, highlighting the main facts and insights."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def email_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are an expert in **email communication**. "
        "Write a professional, polite email draft addressing the given topic. "
        "Use a clear subject line, a greeting, a concise message body, and a proper closing."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def script_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a **scriptwriter**. "
        "Create a short, engaging script suitable for a video or podcast. "
        "Include intro, main talking points, and outro. "
        "Keep it creative and natural in tone."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def social_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a **social media expert**. "
        "Write catchy, platform-appropriate social media content. "
        "Include attention-grabbing hooks, hashtags, and a call to action."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def general_agent(state: MessagesState):
    prompt = SystemMessage(content=(
        "You are a helpful **AI content writer**. "
        "Write a detailed and informative explanation of the topic provided."
    ))
    return {"messages": [llm.invoke([prompt] + state["messages"])]}


# --- Define Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}


# --- Helper Function: Decide Which Agent to Use ---
def select_content_agent(state: MessagesState):
    """
    Decide which content agent to use based on `content_type` in the user's message.
    """
    messages = state["messages"]
    content_type = None

    for msg in messages:
        if isinstance(msg, HumanMessage) and isinstance(msg.content, dict):
            content_type = msg.content.get("content_type", "").lower()
            break

    mapping = {
        "blog": "blog_agent",
        "article": "article_agent",
        "summary": "summary_agent",
        "email": "email_agent",
        "script": "script_agent",
        "social": "social_agent",
    }

    return mapping.get(content_type, "general_agent")


# --- Build Graph ---
graph = StateGraph(MessagesState)
graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))

# Add specialized content agents
graph.add_node("blog_agent", blog_agent)
graph.add_node("article_agent", article_agent)
graph.add_node("summary_agent", summary_agent)
graph.add_node("email_agent", email_agent)
graph.add_node("script_agent", script_agent)
graph.add_node("social_agent", social_agent)
graph.add_node("general_agent", general_agent)
graph.add_node("tune_agent", tune_agent)

# --- Graph Connections ---
graph.add_edge(START, "search_agent")

# Conditional path after search → to tool or next content agent
graph.add_conditional_edges("search_agent", tools_condition, {"tools": "tools", "__end__": select_content_agent})
graph.add_edge("tools", "search_agent")

# Content agents → tuning → END
for node in ["blog_agent", "article_agent", "summary_agent", "email_agent", "script_agent", "social_agent", "general_agent"]:
    graph.add_edge(node, "tune_agent")

graph.add_edge("tune_agent", END)

# --- Compile Final Agent ---
agent = graph.compile()


In [ ]:


import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from myagents.prompts import agent1_prompt, agent2_prompt, agent3_prompt


# --- Initialize LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# llm = ChatOpenAI(model="gpt-4o-mini")

tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}


# === Specialized Content Agents ===
def blog_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write an engaging, SEO-friendly blog post.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def article_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a well-researched, formal article.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def summary_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Summarize the topic in concise bullet points.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def email_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a professional email draft.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def script_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Create a creative video or podcast script.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def social_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a catchy social media caption or post.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}

def general_agent(state: MessagesState):
    prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: Write a detailed explanation.")
    return {"messages": [llm.invoke([prompt] + state["messages"])]}


# --- Define Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}


# --- Helper Function: Decide Which Agent to Use ---
def select_content_agent(state: MessagesState):
    """
    Decide which content agent to use based on `content_type` inside the user's message.
    """
    messages = state["messages"]
    content_type = None

    for msg in messages:
        if isinstance(msg, HumanMessage) and isinstance(msg.content, dict):
            content_type = msg.content.get("content_type", "").lower()
            break

    mapping = {
        "blog": "blog_agent",
        "article": "article_agent",
        "summary": "summary_agent",
        "email": "email_agent",
        "script": "script_agent",
        "social": "social_agent",
    }

    # Default to general_agent if not found
    return mapping.get(content_type, "general_agent")


# --- Build Graph ---
graph = StateGraph(MessagesState)
graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))

# Add all specialized content agents
graph.add_node("blog_agent", blog_agent)
graph.add_node("article_agent", article_agent)
graph.add_node("summary_agent", summary_agent)
graph.add_node("email_agent", email_agent)
graph.add_node("script_agent", script_agent)
graph.add_node("social_agent", social_agent)
graph.add_node("general_agent", general_agent)

graph.add_node("tune_agent", tune_agent)

# --- Graph Connections ---
graph.add_edge(START, "search_agent")

# Conditional path after search
graph.add_conditional_edges("search_agent", tools_condition, {"tools": "tools", "__end__": select_content_agent})
graph.add_edge("tools", "search_agent")

# After chosen content agent → tuning → END
for node in ["blog_agent", "article_agent", "summary_agent", "email_agent", "script_agent", "social_agent", "general_agent"]:
    graph.add_edge(node, "tune_agent")

graph.add_edge("tune_agent", END)

# --- Compile Final Agent ---
agent = graph.compile()




In [ ]:

import os
import sys
from typing import TypedDict, List
from typing_extensions import Annotated
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from myagents.prompts import agent1_prompt, agent2_prompt, agent3_prompt



from tools.search_tool import search_content

# --- Define Custom Graph State ---
# We use a custom state to store the content_type separately from the messages.
class ContentWriterState(TypedDict):
    """
    Represents the state of our multi-agent content writer.
    Uses 'add_messages' reducer for the message history.
    """
    messages: Annotated[List[BaseMessage], add_messages]
    content_type: str # <-- NEW KEY for dynamic agent routing

# --- Initialize LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: ContentWriterState):
    # The search agent uses the tools-enabled LLM
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}

# --- Define Agent 2: Content Agent (Dynamic by type) ---
def content_agent(state: ContentWriterState):
    """
    Dynamically adjusts the content generation style based on content_type 
    extracted directly from the state dictionary.
    """
    messages = state["messages"]
    
    # --- FIX: Extract content_type directly from the state ---
    # This key was added in app.py when calling agent.invoke
    content_type = state.get("content_type", "general").lower()
    
    # Define dynamic content instructions
    style_prompts = {
        "blog": "Write an engaging, SEO-friendly blog post about this topic. Include headings and structure.",
        "article": "Write a well-researched, professional article with a formal tone.",
        "summary": "Summarize the topic in concise, clear bullet points or short paragraphs.",
        "email": "Write a professional email draft addressing this topic.",
        "script": "Create a creative video or podcast script covering this topic.",
        "social": "Write a short, catchy social media caption or post about this topic.",
    }

    style_prompt = style_prompts.get(content_type, "Write a detailed explanation about this topic.")

    # Combine system prompt dynamically (agent2_prompt sets the base persona)
    dynamic_prompt = SystemMessage(content=f"{agent2_prompt}\n\nInstruction: {style_prompt}")

    # Pass modified prompt and the *full* message history to LLM
    return {"messages": [llm.invoke([dynamic_prompt] + messages)]}

# --- Define Agent 3: Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: ContentWriterState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}

# --- Build Graph ---
# FIX: Use the custom state defined above
graph = StateGraph(ContentWriterState) 

graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))
graph.add_node("content_agent", content_agent)
graph.add_node("tune_agent", tune_agent)


# --- Graph Connections ---
graph.add_edge(START, "search_agent")

# If search_agent calls tool, go to tools. If it's done (__end__), go to content_agent.
graph.add_conditional_edges(
    "search_agent",
    tools_condition,
    {"tools": "tools", "__end__": "content_agent"}
)

# After tools run (search results received) → back to search_agent for next step
graph.add_edge("tools", "search_agent")

# Removed redundant edge: graph.add_edge("search_agent", "content_agent")

graph.add_edge("content_agent", "tune_agent")

# tune Agent → END
graph.add_edge("tune_agent", END)

# --- Compile Agent ---
agent = graph.compile()

# # --- Run the flow ---
mesg = HumanMessage(content="What is Adobe Experience Manager?")
result = agent.invoke({"messages": mesg})
print(result["messages"])

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from myagents.prompts import agent1_prompt,agent2_prompt,agent3_prompt
from myagents.prompts import blog_agent_prompt, summary_agent_prompt, tutorial_agent_prompt, comparison_agent_prompt



from tools.search_tool import search_content


llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# llm = ChatOpenAI(model="gpt-4o-mini")
tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

blog_agent_prompt_msg = SystemMessage(content=blog_agent_prompt)
summary_agent_prompt_msg = SystemMessage(content=summary_agent_prompt)
tutorial_agent_prompt_msg = SystemMessage(content=tutorial_agent_prompt)
comparison_agent_prompt_msg = SystemMessage(content=comparison_agent_prompt)

def blog_agent(state: MessagesState):
    return {"messages": [llm.invoke([blog_agent_prompt_msg] + state["messages"])]}

def summary_agent(state: MessagesState):
    return {"messages": [llm.invoke([summary_agent_prompt_msg] + state["messages"])]}

def tutorial_agent(state: MessagesState):
    return {"messages": [llm.invoke([tutorial_agent_prompt_msg] + state["messages"])]}

def comparison_agent(state: MessagesState):
    return {"messages": [llm.invoke([comparison_agent_prompt_msg] + state["messages"])]}






orchestrator_prompt = SystemMessage(
    content="""You are an orchestrator agent. 
Your job is to decide which content type best matches the user request. 
Possible content types are:
- blog
- summary
- tutorial
- comparison

Return the name of the type that fits best (just one word: blog, summary, tutorial, or comparison)."""
)

def orchestrator_agent(state: MessagesState):
    decision = llm.invoke([orchestrator_prompt] + state["messages"])
    decision_text = decision.content.strip().lower()

    if "blog" in decision_text:
        next_node = "blog_agent"
    elif "summary" in decision_text:
        next_node = "summary_agent"
    elif "tutorial" in decision_text:
        next_node = "tutorial_agent"
    elif "comparison" in decision_text:
        next_node = "comparison_agent"
    else:
        next_node = "summary_agent"  # default fallback

    return {"messages": [decision], "next_node": next_node}














# --- Define Agent 1: Search Agent ---
search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}

# --- Define Agent 2: Content Agent ---
content_agent_prompt = SystemMessage(content=agent2_prompt)
def content_agent(state: MessagesState):
    return {"messages": [llm.invoke([content_agent_prompt] + state["messages"])]}
















# --- Define Agent 3: Tuning Agent ---
tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}

# --- Build Graph ---
graph = StateGraph(MessagesState)

graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))
graph.add_node("orchestrator_agent", orchestrator_agent)
graph.add_node("blog_agent", blog_agent)
graph.add_node("summary_agent", summary_agent)
graph.add_node("tutorial_agent", tutorial_agent)
graph.add_node("comparison_agent", comparison_agent)
graph.add_node("tune_agent", tune_agent)

# Flow
graph.add_edge(START, "search_agent")
graph.add_conditional_edges(
    "search_agent",
    tools_condition,
    {"tools": "tools", "__end__": "orchestrator_agent"}
)
graph.add_edge("tools", "search_agent")

# Orchestrator decides next path dynamically
graph.add_conditional_edges(
    "orchestrator_agent",
    lambda state: state["next_node"],
    {
        "blog_agent": "blog_agent",
        "summary_agent": "summary_agent",
        "tutorial_agent": "tutorial_agent",
        "comparison_agent": "comparison_agent"
    }
)

# After content creation → go to tune agent
graph.add_edge("blog_agent", "tune_agent")
graph.add_edge("summary_agent", "tune_agent")
graph.add_edge("tutorial_agent", "tune_agent")
graph.add_edge("comparison_agent", "tune_agent")

graph.add_edge("tune_agent", END)

# --- Compile Agent ---
agent = graph.compile()


agent

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from agents.prompts import web_copy_prompt
from agents.prompts import (
    agent1_prompt,
    agent2_prompt,
    agent3_prompt,
    thought_leadership_prompt,
    web_copy_prompt,
    email_sequence_prompt,
    social_post_prompt,
    sales_enablement_prompt,
    blog_post_prompt
)

from tools.search_tool import search_content

# --- Load environment variables ---
load_dotenv()

# --- Initialize LLM ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# Alternative if you want to test with OpenAI:
# llm = ChatOpenAI(model="gpt-4o-mini")

tools = [search_content]
llm_with_tools = llm.bind_tools(tools)

# ============================================================
# 🧠 DEFINE INDIVIDUAL CONTENT AGENTS
# ============================================================

def thought_leadership_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=thought_leadership_prompt)] + state["messages"])]}

def web_copy_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=web_copy_prompt)] + state["messages"])]}

def email_sequence_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=email_sequence_prompt)] + state["messages"])]}

def social_post_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=social_post_prompt)] + state["messages"])]}

def sales_enablement_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=sales_enablement_prompt)] + state["messages"])]}

def blog_post_agent(state: MessagesState):
    return {"messages": [llm.invoke([SystemMessage(content=blog_post_prompt)] + state["messages"])]}

# ============================================================
# 🧭 ORCHESTRATOR
# ============================================================

orchestrator_prompt = SystemMessage(content="""
You are an orchestrator agent.
Your job is to decide which content type best fits the user’s intent.
Available content types:
- thought_leadership
- web_copy
- email_sequence
- social_post
- sales_enablement
- blog_post

Return ONLY one keyword exactly matching the best type.
""")

def orchestrator_agent(state: MessagesState):
    decision = llm.invoke([orchestrator_prompt] + state["messages"])
    decision_text = decision.content.strip().lower()

    valid_types = [
        "thought_leadership", "web_copy", "email_sequence",
        "social_post", "sales_enablement", "blog_post"
    ]
    next_node = decision_text if decision_text in valid_types else "blog_post"  # fallback

    print(f"🧭 Orchestrator chose: {next_node}")
    return {"messages": [decision], "next_node": next_node}

# ============================================================
# 🔍 SEARCH AGENT + TOOLS + TUNING AGENT
# ============================================================

search_agent_prompt = SystemMessage(content=agent1_prompt)
def search_agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([search_agent_prompt] + state["messages"])]}

content_agent_prompt = SystemMessage(content=agent2_prompt)
def content_agent(state: MessagesState):
    return {"messages": [llm.invoke([content_agent_prompt] + state["messages"])]}

tune_agent_prompt = SystemMessage(content=agent3_prompt)
def tune_agent(state: MessagesState):
    return {"messages": [llm.invoke([tune_agent_prompt] + state["messages"])]}

# ============================================================
# 🕸️ BUILD THE GRAPH
# ============================================================

graph = StateGraph(MessagesState)

# --- Add nodes ---
graph.add_node("search_agent", search_agent)
graph.add_node("tools", ToolNode(tools))
graph.add_node("orchestrator_agent", orchestrator_agent)
graph.add_node("tune_agent", tune_agent)

# --- Add all content-type agents ---
graph.add_node("thought_leadership", thought_leadership_agent)
graph.add_node("web_copy", web_copy_agent)
graph.add_node("email_sequence", email_sequence_agent)
graph.add_node("social_post", social_post_agent)
graph.add_node("sales_enablement", sales_enablement_agent)
graph.add_node("blog_post", blog_post_agent)

# --- Flow Setup ---
graph.add_edge(START, "search_agent")

graph.add_conditional_edges(
    "search_agent",
    tools_condition,
    {"tools": "tools", "__end__": "orchestrator_agent"}
)
graph.add_edge("tools", "search_agent")

# Orchestrator → correct agent
graph.add_conditional_edges(
    "orchestrator_agent",
    lambda state: state["next_node"],
    {
        "thought_leadership": "thought_leadership",
        "web_copy": "web_copy",
        "email_sequence": "email_sequence",
        "social_post": "social_post",
        "sales_enablement": "sales_enablement",
        "blog_post": "blog_post"
    }
)

# After generation → Tune → END
for node in ["thought_leadership", "web_copy", "email_sequence", "social_post", "sales_enablement", "blog_post"]:
    graph.add_edge(node, "tune_agent")

graph.add_edge("tune_agent", END)

# --- Compile the graph ---
agent = graph.compile()
agent